# Counting and aggregating M&M


In [3]:
# Import the necessary libraries.
# Since we are using Python, import the SparkSession and related functions from the Pyspark module.
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

In [4]:
# Build a SparkSession using the SparkSession APIs.
# If one does not exist, then create an instance.
# There can only be one SparkSession per JVM.
spark = (SparkSession
        .builder
        .appName("PythonMnMCount")
        .getOrCreate())

In [5]:
# Get the M&M data set filename from the command-line arguments.
mnm_file = "C:/Users/mariajose.chinchilla/OneDrive - Bosonit/Escritorio/Bosonit/Spark/datarepositorio/chapter2/py/src/data/mnm_dataset.csv"

In [6]:
# Read the file into a Spark DataFrame using the CSV format 
# by inferring the schema and specifying that the file contains a header, 
# which provides column names for comma-separated fields.
mnm_df = (spark.read.format("csv")
         .option("header", "true")
         .option("inferSchema", "true")
         .load(mnm_file))

# Practicando

In [7]:
#Mostramos los datos que hemos cargado
mnm_df.show()

+-----+------+-----+
|State| Color|Count|
+-----+------+-----+
|   TX|   Red|   20|
|   NV|  Blue|   66|
|   CO|  Blue|   79|
|   OR|  Blue|   71|
|   WA|Yellow|   93|
|   WY|  Blue|   16|
|   CA|Yellow|   53|
|   WA| Green|   60|
|   OR| Green|   71|
|   TX| Green|   68|
|   NV| Green|   59|
|   AZ| Brown|   95|
|   WA|Yellow|   20|
|   AZ|  Blue|   75|
|   OR| Brown|   72|
|   NV|   Red|   98|
|   WY|Orange|   45|
|   CO|  Blue|   52|
|   TX| Brown|   94|
|   CO|   Red|   82|
+-----+------+-----+
only showing top 20 rows



In [8]:
#Seleccionamos la columna de estado y color donde el color es rojo
prueba_df = mnm_df.select("State", "Color").filter(col("Color")== "Red")
prueba_df.show()

+-----+-----+
|State|Color|
+-----+-----+
|   TX|  Red|
|   NV|  Red|
|   CO|  Red|
|   CO|  Red|
|   CO|  Red|
|   NV|  Red|
|   WA|  Red|
|   WY|  Red|
|   WA|  Red|
|   UT|  Red|
|   CA|  Red|
|   UT|  Red|
|   WA|  Red|
|   AZ|  Red|
|   CA|  Red|
|   CO|  Red|
|   OR|  Red|
|   CO|  Red|
|   CA|  Red|
|   WA|  Red|
+-----+-----+
only showing top 20 rows



In [9]:
#Cuento los registros 
mnm_df.count()


99999

In [10]:
#Tomo los 10 primeros
mnm_df.show(10)

+-----+------+-----+
|State| Color|Count|
+-----+------+-----+
|   TX|   Red|   20|
|   NV|  Blue|   66|
|   CO|  Blue|   79|
|   OR|  Blue|   71|
|   WA|Yellow|   93|
|   WY|  Blue|   16|
|   CA|Yellow|   53|
|   WA| Green|   60|
|   OR| Green|   71|
|   TX| Green|   68|
+-----+------+-----+
only showing top 10 rows



# Ejercicio

In [13]:
# We use the DataFrame high-level APIs. Note that we don't use RDDs at all. 
# Because some of Spark's functions return the same object, we can chain functions calls.
# 1. Select from the DataFrame the fields "State", "Color", and "Count".
# 2. Since we want to group each state and its M&M color count, we use groupBy()
# 3. Aggregate counts of all colors and groupBy() State and Color
# 4. orderBy() in descending order
count_mnm_df = (mnm_df
               .select("State", "Color", "Count")
               .groupBy("State", "Color")
               .agg(count("Count").alias("Total")) 
               .orderBy("Total", ascending= False)
                )


In [14]:
# Show the resulting aggregations for all the states and colors; a total count of each color per state.
# Note show() is an action, which will trigger the above query to be executed.
count_mnm_df.show(n=60, truncate=False)
print("Total Rows = %d" % (count_mnm_df.count()))

+-----+------+-----+
|State|Color |Total|
+-----+------+-----+
|CA   |Yellow|1807 |
|WA   |Green |1779 |
|OR   |Orange|1743 |
|TX   |Green |1737 |
|TX   |Red   |1725 |
|CA   |Green |1723 |
|CO   |Yellow|1721 |
|CA   |Brown |1718 |
|CO   |Green |1713 |
|NV   |Orange|1712 |
|TX   |Yellow|1703 |
|NV   |Green |1698 |
|AZ   |Brown |1698 |
|WY   |Green |1695 |
|CO   |Blue  |1695 |
|NM   |Red   |1690 |
|AZ   |Orange|1689 |
|NM   |Yellow|1688 |
|NM   |Brown |1687 |
|UT   |Orange|1684 |
|NM   |Green |1682 |
|UT   |Red   |1680 |
|AZ   |Green |1676 |
|NV   |Yellow|1675 |
|NV   |Blue  |1673 |
|WA   |Red   |1671 |
|WY   |Red   |1670 |
|WA   |Brown |1669 |
|NM   |Orange|1665 |
|WY   |Blue  |1664 |
|WA   |Yellow|1663 |
|WA   |Orange|1658 |
|NV   |Brown |1657 |
|CA   |Orange|1657 |
|CA   |Red   |1656 |
|CO   |Brown |1656 |
|UT   |Blue  |1655 |
|AZ   |Yellow|1654 |
|TX   |Orange|1652 |
|AZ   |Red   |1648 |
|OR   |Blue  |1646 |
|UT   |Yellow|1645 |
|OR   |Red   |1645 |
|CO   |Orange|1642 |
|TX   |Brown 

## EXTRA TEST. Explain method
Como ejercicio extra investigamos sobre el plan lógico que se sigue
haciendo uso de la API de Dataframe y de SparkSQL para poder comparar:

In [15]:
count_mnm_df.explain(True)

== Parsed Logical Plan ==
'Sort ['Total DESC NULLS LAST], true
+- Aggregate [State#17, Color#18], [State#17, Color#18, count(Count#19) AS Total#125L]
   +- Project [State#17, Color#18, Count#19]
      +- Relation [State#17,Color#18,Count#19] csv

== Analyzed Logical Plan ==
State: string, Color: string, Total: bigint
Sort [Total#125L DESC NULLS LAST], true
+- Aggregate [State#17, Color#18], [State#17, Color#18, count(Count#19) AS Total#125L]
   +- Project [State#17, Color#18, Count#19]
      +- Relation [State#17,Color#18,Count#19] csv

== Optimized Logical Plan ==
Sort [Total#125L DESC NULLS LAST], true
+- Aggregate [State#17, Color#18], [State#17, Color#18, count(Count#19) AS Total#125L]
   +- Relation [State#17,Color#18,Count#19] csv

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [Total#125L DESC NULLS LAST], true, 0
   +- Exchange rangepartitioning(Total#125L DESC NULLS LAST, 200), ENSURE_REQUIREMENTS, [plan_id=380]
      +- HashAggregate(keys=[State#17, Color#18]

In [17]:
mnm_df.createOrReplaceTempView("mnm_table")

spark.sql("""SELECT State, Color, COUNT(Count) AS Total 
FROM mnm_table
GROUP BY State, Color
ORDER BY Total DESC""").explain(True)

== Parsed Logical Plan ==
'Sort ['Total DESC NULLS LAST], true
+- 'Aggregate ['State, 'Color], ['State, 'Color, 'COUNT('Count) AS Total#160]
   +- 'UnresolvedRelation [mnm_table], [], false

== Analyzed Logical Plan ==
State: string, Color: string, Total: bigint
Sort [Total#160L DESC NULLS LAST], true
+- Aggregate [State#17, Color#18], [State#17, Color#18, count(Count#19) AS Total#160L]
   +- SubqueryAlias mnm_table
      +- View (`mnm_table`, [State#17,Color#18,Count#19])
         +- Relation [State#17,Color#18,Count#19] csv

== Optimized Logical Plan ==
Sort [Total#160L DESC NULLS LAST], true
+- Aggregate [State#17, Color#18], [State#17, Color#18, count(Count#19) AS Total#160L]
   +- Relation [State#17,Color#18,Count#19] csv

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [Total#160L DESC NULLS LAST], true, 0
   +- Exchange rangepartitioning(Total#160L DESC NULLS LAST, 200), ENSURE_REQUIREMENTS, [plan_id=403]
      +- HashAggregate(keys=[State#17, Color#18], function

## Ejercicio.
Continuamos con el ejercicio anterior. 

In [19]:
# While the above code aggregated and counted for all the states, what if we just want to see the data for a single state, e.g., CA?
# 1. Select from all rows in the DataFrame
# 2. Filter only CA state
# 3. groupBy() State and Color as we did above
# 4. Aggregate the counts for each color
# 5. orderBy() in descending order
# Find the aggregate count for California by filtering
ca_count_mnm_df = (mnm_df
               .select("State", "Color", "Count")
               .where(mnm_df.State == "CA")
               .groupBy("State", "Color")
               .agg(count("Count").alias("Total")) 
               .orderBy("Total", ascending= False)
                )


In [20]:
# Show the resulting aggregation for California.
# As above, show() is an action that will trigger the execution of the entire computation
ca_count_mnm_df.show(n=10, truncate=False)

+-----+------+-----+
|State|Color |Total|
+-----+------+-----+
|CA   |Yellow|1807 |
|CA   |Green |1723 |
|CA   |Brown |1718 |
|CA   |Orange|1657 |
|CA   |Red   |1656 |
|CA   |Blue  |1603 |
+-----+------+-----+



In [21]:
#Podemos repetirlo para cada uno de los estados:
tx_count_mnm_df = (mnm_df
               .select("State", "Color", "Count")
               .where(mnm_df.State == "TX")
               .groupBy("State", "Color")
               .agg(count("Count").alias("Total")) 
               .orderBy("Total", ascending= False)
                )
tx_count_mnm_df.show(n=10, truncate=False)

+-----+------+-----+
|State|Color |Total|
+-----+------+-----+
|TX   |Green |1737 |
|TX   |Red   |1725 |
|TX   |Yellow|1703 |
|TX   |Orange|1652 |
|TX   |Brown |1641 |
|TX   |Blue  |1614 |
+-----+------+-----+



In [22]:
nv_avg_mnm_df = (mnm_df
    .select("State", "Color", "Count")
    .where(mnm_df.State == "NV")
    .groupBy("State", "Color")
    .agg(avg("Count").alias("avg"))
    .orderBy("avg", ascending=False))

nv_avg_mnm_df.show(n=10, truncate=False)

+-----+------+------------------+
|State|Color |avg               |
+-----+------+------------------+
|NV   |Brown |55.81050090525045 |
|NV   |Red   |55.4944099378882  |
|NV   |Orange|54.865070093457945|
|NV   |Yellow|54.561194029850746|
|NV   |Blue  |53.797369994022716|
|NV   |Green |53.78739693757362 |
+-----+------+------------------+

